In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as skl
import seaborn as sns

In [2]:
%cd /home/stepan/kaggle/cars_from_auction

/home/stepan/kaggle/cars_from_auction


In [3]:
%ls

data/  models/  prepare/  result/  review/  test/  train/


In [4]:
%ls data/

processed/  sample_submission.csv  test.csv  train.csv  zipcodes.csv


In [5]:
data_path = 'data/processed/'
data_file = 'transformed_test.csv'

In [6]:
data = pd.read_csv(data_path + data_file, index_col=0)

In [7]:
data

,engine_capacity,type,registration_year,gearbox,power,model,mileage,fuel,brand,damage,zipcode,insurance_price
60314,1.6,small car,2013,manual,136.0,swift,40000,gasoline,suzuki,0.0,30449,6.194405
12566,NaN,coupé,2004,auto,333.0,6er,150000,gasoline,bmw,0.0,45307,6.507278
17760,NaN,station wagon,2006,auto,170.0,e_klasse,150000,diesel,mercedes_benz,0.0,59494,6.131226
8876,NaN,limousine,1999,manual,101.0,astra,150000,gasoline,opel,NaN,25524,NaN
80392,NaN,limousine,1975,manual,54.0,andere,150000,diesel,mercedes_benz,0.0,70794,7.012115
...,...,...,...,...,...,...,...,...,...,...,...,...
93878,1.4,limousine,1999,manual,86.0,corolla,150000,gasoline,toyota,0.0,44339,4.700480
99783,NaN,station wagon,2002,auto,184.0,3er,150000,diesel,bmw,0.0,47574,5.480639
57399,NaN,small car,2005,manual,52.0,fox,100000,gasoline,volkswagen,0.0,50389,4.094345
97106,NaN,bus,2001,manual,151.0,transporter,150000,diesel,volkswagen,0.0,12209,6.835185


In [8]:
data.describe()

,engine_capacity,registration_year,power,mileage,damage,zipcode,insurance_price
count,20086.000000,50000.000000,50000.000000,50000.000000,41773.000000,50000.000000,42594.000000
mean,1.780643,2003.733600,122.107725,125278.300000,0.092931,51196.608000,5.479055
std,0.470688,7.299557,60.319161,39509.609835,0.290339,25750.460527,1.042473
min,0.800000,1945.000000,25.000000,5000.000000,0.000000,1067.000000,2.302585
25%,1.400000,2000.000000,75.000000,100000.000000,0.000000,30926.000000,4.605170
50%,1.800000,2004.000000,110.000000,150000.000000,0.000000,50170.000000,5.438079
75%,2.000000,2008.000000,150.000000,150000.000000,0.000000,71720.750000,6.234411
max,3.300000,2016.000000,495.000000,150000.000000,1.000000,99998.000000,10.858807


In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50000 entries, 60314 to 478
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   engine_capacity    20086 non-null  float64
 1   type               43834 non-null  object 
 2   registration_year  50000 non-null  int64  
 3   gearbox            48021 non-null  object 
 4   power              50000 non-null  float64
 5   model              47671 non-null  object 
 6   mileage            50000 non-null  int64  
 7   fuel               46408 non-null  object 
 8   brand              50000 non-null  object 
 9   damage             41773 non-null  float64
 10  zipcode            50000 non-null  int64  
 11  insurance_price    42594 non-null  float64
dtypes: float64(4), int64(3), object(5)
memory usage: 5.0+ MB


# Fill

In [10]:
from sklearn.linear_model import LinearRegression

In [11]:
def fill_engine(d):
    d = d.copy()
    engine = d.dropna(subset=['engine_capacity'])
    
    X_train, y_train = engine['power'], engine['engine_capacity']
    X_train = X_train.values.reshape(-1, 1)
    reg = LinearRegression().fit(X_train, y_train)
    
    all_engine = d['engine_capacity']
    
    X = d[all_engine.isna()]['power']
    X = X.values.reshape(-1, 1)
    d.loc[all_engine.isna(), 'engine_capacity'] = reg.predict(X)
    return d

In [12]:
def fill_insurance(d):
    d = d.copy()
    d.loc[d['insurance_price'].isna(), 'insurance_price'] = d['insurance_price'].median()
    return d

In [13]:
def fill_damage(d):
    d = d.copy()
    d.loc[d['damage'].isna(), 'damage'] = d['damage'].mode()[0]
    return d

In [14]:
def fill_type(d):
    d = d.copy()
    d.loc[d['type'].isna(), 'type'] = d['type'].mode()[0]
    return d

In [15]:
def fill_model(d):
    d = d.copy()
    d.loc[d['model'].isna(), 'model'] = d['model'].mode()[0]
    return d

In [16]:
def fill_gearbox(d):
    d = d.copy()
    d.loc[d['gearbox'].isna(), 'gearbox'] = d['gearbox'].mode()[0]
    return d

In [17]:
def fill_fuel(d):
    d = d.copy()
    d.loc[d['fuel'].isna(), 'fuel'] = d['fuel'].mode()[0]
    return d

## Apply

In [18]:
fillers = [fill_engine, fill_insurance, fill_damage, fill_type, fill_model, fill_gearbox, fill_fuel]

In [19]:
new_data = data
for f in fillers:
    new_data = f(new_data)

In [20]:
new_data.describe()

,engine_capacity,registration_year,power,mileage,damage,zipcode,insurance_price
count,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000
mean,1.800151,2003.733600,122.107725,125278.300000,0.077640,51196.608000,5.472985
std,0.441822,7.299557,60.319161,39509.609835,0.267607,25750.460527,0.962283
min,0.800000,1945.000000,25.000000,5000.000000,0.000000,1067.000000,2.302585
25%,1.434372,2000.000000,75.000000,100000.000000,0.000000,30926.000000,4.787492
50%,1.761031,2004.000000,110.000000,150000.000000,0.000000,50170.000000,5.438079
75%,2.000000,2008.000000,150.000000,150000.000000,0.000000,71720.750000,6.086775
max,4.188554,2016.000000,495.000000,150000.000000,1.000000,99998.000000,10.858807


In [21]:
new_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50000 entries, 60314 to 478
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   engine_capacity    50000 non-null  float64
 1   type               50000 non-null  object 
 2   registration_year  50000 non-null  int64  
 3   gearbox            50000 non-null  object 
 4   power              50000 non-null  float64
 5   model              50000 non-null  object 
 6   mileage            50000 non-null  int64  
 7   fuel               50000 non-null  object 
 8   brand              50000 non-null  object 
 9   damage             50000 non-null  float64
 10  zipcode            50000 non-null  int64  
 11  insurance_price    50000 non-null  float64
dtypes: float64(4), int64(3), object(5)
memory usage: 5.0+ MB


In [22]:
new_data.to_csv(data_path + 'filled_' + data_file)